In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Data
data = DataLoader(datasets.MNIST('.', train=True, download=True,
                                 transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])),
                  batch_size=64, shuffle=True)

# Generator & Discriminator
G = nn.Sequential(nn.Linear(100, 256), nn.ReLU(), nn.Linear(256, 784), nn.Tanh())
D = nn.Sequential(nn.Linear(784, 256), nn.LeakyReLU(0.2), nn.Linear(256, 1), nn.Sigmoid())
opt_G = optim.Adam(G.parameters(), lr=0.0002)
opt_D = optim.Adam(D.parameters(), lr=0.0002)
loss = nn.BCELoss()

# Train
for epoch in range(5): # few epochs for quick training
  for real, _ in data:
    real = real.view(-1, 784)
    z = torch.randn(real.size(0), 100)
    fake = G(z)
    # Discriminator
    D_real = D(real)
    D_fake = D(fake.detach())
    loss_D = loss(D_real, torch.ones_like(D_real)) + loss(D_fake, torch.zeros_like(D_fake))
    opt_D.zero_grad(); loss_D.backward(); opt_D.step()

    # Generator
    D_fake = D(fake)
    loss_G = loss(D_fake, torch.ones_like(D_fake))
    opt_G.zero_grad(); loss_G.backward(); opt_G.step()
  print(f"Epoch {epoch+1}: D_loss={loss_D.item():.3f}, G_loss={loss_G.item():.3f}")

# Generate sample
z = torch.randn(1, 100)
img = G(z).view(28, 28).detach()
plt.imshow(img, cmap='gray'); plt.axis('off'); plt.show()